Идея проекта заключается в том, чтобы посмотреть, каким образом будет кластеризоваться набор довольно случайных pdf-туториалов по вязанию.

- **Задача**: кластеризовать пдфки

а) по их текстовому содержанию;

б) по картинкам, которые в них вставлены.

- **Данные**: pdf-файлы, написанные по-английски. Файлы распределены по папкам в соответствии с тем, какой тип одежды там описан (свитера, носки, шарфы+шапки+перчатки+иное, изделия крючком)

Данных не очень много (127 файлов в общей сложности), так что кластеризацию я буду проводить вслепую, без обучения. После кластеризации я хочу посмотреть, что больше повлияло на кластеры -- вид изделия, дизайнер или какие-то особенности конкретных пдфок.



#Подготовка

In [ ]:
try:
    from pypdf import PdfReader
except:
    !pip install pypdf
    from pypdf import PdfReader
    !pip install nltk
    !pip install -U tqdm

In [ ]:
import logging

logger = logging.getLogger("pypdf")
logger.setLevel(logging.ERROR)

In [ ]:
import os
import random
import pandas as pd
from google.colab import files
import csv
import numpy as np

# для текстов
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from string import punctuation
import nltk
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import time

#для картинок
from google.colab.patches import cv2_imshow
import cv2
from PIL import Image
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#путь к папке с вязанием
os.chdir('/content/drive/MyDrive/дашины книги по вязанию/вязание')

# 1. Классификация по текстовому наполнению
#### 1.1. Парсинг текстов из pdf файлов

In [ ]:
# функция открывает пдфку и достаёт оттуда текст
# лемматизирует и убирает знаки препинания

stops = stopwords.words('english')

def text_reader(address):
  reader = PdfReader(address)
  out = list()
  for page in reader.pages:
      out.append(page.extract_text())
  raw_text = ' '.join(out)
  clean_text = word_tokenize(raw_text.lower())
  clean_text = list(filter(lambda x: (x not in punctuation) & (x not in ['\n', '–', '«', '»'])
                & (x not in stops), clean_text))
  wnl = WordNetLemmatizer()
  lemm_text = [wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i) for i,j in pos_tag(clean_text)]
  return ' '.join(lemm_text)

In [ ]:
# пдфки делятся условно на такие темы
knit_dirs = next(os.walk('.'))[1]
knit_dirs

In [ ]:
# для экономии времени загрузки я вручную исключу папки с книгами и журналами, т.к. они очень тяжёлые
# на общую идею повлиять не должно
knit_dirs.remove('книги')
knit_dirs.remove('журналы')

if 'картинки' in knit_dirs:
    knit_dirs.remove('картинки')
knit_dirs

In [ ]:
knits = pd.DataFrame(columns=['label', 'category', 'text'])
for dir in tqdm(knit_dirs):
    for file in tqdm(os.listdir(dir), leave=False):
        tekst = text_reader(os.path.join(dir, file))
        knits = pd.concat([pd.DataFrame([[file, dir, tekst]], columns=knits.columns), knits], ignore_index=True)


In [ ]:
knits.head()

### 1.2. Векторизация текстов и кластеризация через k-means

NB: количество кластеров я задам вручную как 4

In [ ]:
pdf_texts = knits['text']
vectorizer = TfidfVectorizer(stop_words='english')
vectorized_documents = vectorizer.fit_transform(pdf_texts)
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(vectorized_documents.toarray())

In [ ]:
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, n_init=5, max_iter=500, random_state=42)
kmeans.fit(vectorized_documents)

In [ ]:
reduced_data.shape
knits.shape

In [ ]:
knits['text_cluster'] = kmeans.labels_

knits = pd.concat((knits, pd.DataFrame(reduced_data)), axis=1)

In [ ]:
knits.columns

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(221)
colors = ['orange', 'navy', 'yellow', 'purple']
cluster_labels = knit_dirs
for i in range(num_clusters):
    plt.scatter(knits.loc[knits['text_cluster'] == i, 0],
                knits.loc[knits['text_cluster'] == i, 1],
                s=10, color=colors[i],
                label=f'{cluster_labels[i]}')


colors = ['maroon', 'green', 'blue', 'pink']
ax2 = fig.add_subplot(222)
real_cats = knits['category']
for i, label in enumerate(knit_dirs):
    plt.scatter(knits.loc[knits['category'] == label, 0],
                knits.loc[knits['category'] == label, 1],
                s=10, color=colors[i],
                label=f'{cluster_labels[i]}', marker='h', alpha = 1)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))


ax1.set_title('Кластеризация')
ax2.set_title('Реальные группы')

plt.show()

### 1.3. Что скластеризовалось?

In [ ]:
# на графике с реальными категориями вообще не видно крючка
# это ожидаемо, т.к. крючковые схемы не содержат текста а просто узор, см. отсутствие текста ниже

knits[knits['category']=='крючок']

In [ ]:
# посмотрим, какие характеристики скластеризовал kmeans
knits.loc[knits['text_cluster']==1, 'label'].values[:30]

# одна из категорий в основном собрана из дизайнеров PetiteKnit, Ozetta и MyFavouriteThings

In [ ]:
# ещё одна категория угадала носки
knits.loc[knits['text_cluster']==2]

In [ ]:
# а другая исключительно состоит из работ одного дизайнера -- Stephen West
# при этом пара его работ проскочила в категорию 0
knits.loc[knits['text_cluster']==3]

# 2. Классифицируем по картинкам

### 2.1. Вытаскиваем картинки

In [ ]:
reader = PdfReader("./свитера/Afra-sweater.pdf")

def pic_shower(f_name):
    img = cv2.imread(f_name)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(image)


page = reader.pages[0]

for i, image_file_object in enumerate(page.images):
    file_name = "out-image-" + str(i) + "-" + image_file_object.name
    image_file_object.image.save(file_name)
    #print(image_file_object.image.size)
    pic_shower(file_name)

In [ ]:
os.mkdir('картинки')

In [ ]:
#функция для вытаскивания картинок
# в рамках лени будем смотреть на картинки только с первой страницы


# чтобы не вытаскивать декоративные элементы, буду выбирать самую большую картинку на странице
# это с большей вероятностью будет изображение самого изделия
def get_img_size(path):
    width, height = path.image.size
    return width*height

def image_grabber(file_path):
    reader = PdfReader(file_path)
    page = reader.pages[0]
    largest = max(page.images, key=get_img_size)
    label = os.path.splitext(os.path.split(file_path)[1])[0]
    file_name = label + "-" + largest.name
    largest.image.save(os.path.join('картинки', file_name))
    return file_name

In [ ]:
knits['pics'] = knits.apply(lambda x: image_grabber(os.path.join(x.category, x.label)), axis=1)

In [ ]:
knits.head()

### 2.2. Векторизация и кластеризация картинок

In [ ]:
#image.LOAD_TRUNCATED_IMAGES = True
model = VGG16(weights='imagenet', include_top=False)

num_clusters = 4

# Loop over files and get features
filelist = knits.apply(lambda x: os.path.join('картинки', x.pics), axis=1).values
#filelist.sort()
featurelist = []
for i, imagepath in enumerate(filelist):
    print("    Status: %s / %s" %(i, len(filelist)), end="\r")
    img = image.load_img(imagepath, target_size=(1024,1024))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = np.array(model.predict(img_data))
    featurelist.append(features.flatten())

# Clustering
kmeans_img = KMeans(n_clusters=num_clusters, random_state=42).fit(np.array(featurelist))

knits['img_cluster'] = kmeans_img.labels_

In [ ]:
# делаем векторы двумерными для графика
pca = PCA(n_components=2)
img_reduced_data = pca.fit_transform(np.array(featurelist))
knits = pd.concat((knits, pd.DataFrame(img_reduced_data, columns= ['img_0', 'img_1'])), axis=1)

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(221)
colors = ['orange', 'navy', 'yellow', 'purple']
cluster_labels = knit_dirs
for i in range(num_clusters):
    plt.scatter(knits.loc[knits['img_cluster'] == i, 'img_0'],
                knits.loc[knits['img_cluster'] == i, 'img_1'],
                s=10, color=colors[i],
                label=f'{cluster_labels[i]}')


colors = ['maroon', 'green', 'blue', 'pink']
ax2 = fig.add_subplot(222)
real_cats = knits['category']
for i, label in enumerate(knit_dirs):
    plt.scatter(knits.loc[knits['category'] == label, 'img_0'],
                knits.loc[knits['category'] == label, 'img_1'],
                s=10, color=colors[i],
                label=f'{cluster_labels[i]}', marker='h', alpha = 1)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))


ax1.set_title('Кластеризация')
ax2.set_title('Реальные группы')

plt.show()

### 2.3. Что скластеризовалось?

In [ ]:

knits[knits['img_cluster'] == 0]

In [ ]:
# первая группа получилась рандомная
pic = knits.loc[knits['img_cluster'] == 0, 'pics'].values[1]
img = os.path.join('картинки', pic)
pic_shower(img)

In [ ]:
knits[knits['img_cluster'] == 2]

In [ ]:
# во вторую группу выделилось одно очень серое изображение
pic = knits.loc[knits['img_cluster'] == 2, 'pics'].values[0]
img = os.path.join('картинки', pic)
pic_shower(img)

In [ ]:
knits[knits['img_cluster'] == 3]

In [ ]:
# в третью группу попало всё подряд
pic = knits.loc[knits['img_cluster'] == 3, 'pics'].values[4]
img = os.path.join('картинки', pic)
pic_shower(img)

In [ ]:
#а в четвёртую попали случайно просочившиеся лейблы или элементы дизайна как ниже
pic = knits.loc[knits['img_cluster'] == 1, 'pics'].values[0]
img = os.path.join('картинки', pic)
pic_shower(img)

In [ ]:
knits.shape